## Extracción de caracteristica Color

In [116]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from skimage import data, img_as_float
from skimage.segmentation import chan_vese
import os
import xlsxwriter
from scipy.spatial import distance

In [ ]:
# Acceso al directorio
directorio='C:/Users/Acer/Documents/Seminario/Shell_env - Editado/data_img'
img_names=os.listdir(directorio)

#Creando archivo xlsx
workbook = xlsxwriter.Workbook('color_feature.xlsx')
worksheet = workbook.add_worksheet('color_feature')

# Extracción de características
row_hist=0
for i in range(len(img_names)):
    image_path = directorio + '/' + img_names[i]
    img=cv2.imread(image_path)

    # Create a copy
    img2=img.copy()

    # Create empty mask.
    BW = np.zeros((img2.shape[0],img2.shape[1]),np.uint8)

    # Flood fill
    row=5
    column=5

    gray=cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY) #conversión a escala de grises
    ret,th=cv2.threshold(gray,30,255,cv2.THRESH_BINARY_INV) #binarizado inverso
    inv_th=cv2.bitwise_not(th)

    th2=inv_th.copy()
    h, w=th2.shape
    mask = np.zeros((h+2,w+2),np.uint8)

    cv2.floodFill(th2,mask,(row,column),255)
    inv=cv2.bitwise_not(th2)

    BW = inv_th|inv

    #Extraccion de frecuencias del histograma RGB
    b = cv2.calcHist([img2], [0], BW, [256], [0, 256])
    g = cv2.calcHist([img2], [1], BW, [256], [0, 256])
    r = cv2.calcHist([img2], [2], BW, [256], [0, 256])

    vect=[r,g,b]

    for a in vect:
        col_hist=0
        for b in range(len(a)):
            worksheet.write(row_hist, col_hist, a[b])
            col_hist+=1
        worksheet.write(row_hist, col_hist, img_names[i])
        row_hist+=1

        
workbook.close()


#cv2.imshow("Image", img2)
#cv2.waitKey(0)
#cv2.destroyAllWindows()


## Extracción característica Forma

In [ ]:
# Acceso al directorio
directorio='C:/Users/Acer/Documents/Seminario/Shell_env - Editado/data_img'
img_names=os.listdir(directorio)

#Creando archivo xlsx
workbook = xlsxwriter.Workbook('form_feature.xlsx')
worksheet = workbook.add_worksheet('form_feature')

#Extraccion de característica Forma
row_hist=0
for shell in range(len(img_names)):
  image_path = directorio + '/' + img_names[shell]
  img=cv2.imread(image_path)

  # Create a copy
  img2=img.copy()

  # Create empty mask.
  BW = np.zeros((img2.shape[0],img2.shape[1]),np.uint8)

  # Flood fill
  row=5
  column=5

  gray=cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY) #conversión a escala de grises
  ret,th=cv2.threshold(gray,30,255,cv2.THRESH_BINARY_INV) #binarizado inverso
  inv_th=cv2.bitwise_not(th)

  th2=inv_th.copy()
  h, w=th2.shape
  mask = np.zeros((h+2,w+2),np.uint8)

  cv2.floodFill(th2,mask,(row,column),255)
  inv=cv2.bitwise_not(th2)

  BW = inv_th|inv

  #Activando contorno
  cv = chan_vese(gray, mu=0.25, lambda1=1, lambda2=2, tol=1e-3,
                max_iter=50, dt=0.5, init_level_set="checkerboard",
                extended_output=True)

  c_v = np.zeros((cv[0].shape[0],cv[0].shape[1]),np.uint8)

  for i in range(cv[0].shape[0]):
    for j in range(cv[0].shape[1]):
      if cv[0][i,j]==True:
        c_v[i,j]=1
      else:
        c_v[i,j]=0

  contornos,jerarquia=cv2.findContours(c_v,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

  #Unión de puntos
  n=len(contornos)
  if (n==1):
    cnt=np.array(contornos[0])
  else:
    largo=0
    for a in range(n):
        largo+=len(contornos[a])

    nn=largo
    m=2
    contornos_new=[[0]*m]*nn

    pos=0
    for b in range(n):
        for c in range(len(contornos[b])):
            contornos_new[pos]=contornos[b][c]
            pos+=1
    cnt=np.array(contornos_new)

  # Momentos de una imagen → importantes M00-M10-M01
  M=cv2.moments(cnt)

  #Cálculo del centroide
  cX=int(M["m10"]/M["m00"])
  cY=int(M["m01"]/M["m00"])

  a = (cX,cY)
  n=len(cnt)
  angles=np.zeros(n)
  distances=np.zeros(n)
  for i in range(n):
    angles[i]=np.arctan((cnt[i][0][1]-cY)/(cnt[i][0][0]-cX))
    distances[i]=distance.euclidean(a,cnt[i])

  new_angles=np.zeros(n)
  for i in range(n):
    if(cnt[i][0][0]<a[0] and cnt[i][0][1]<a[1]):
      new_angles[i]=180-np.degrees(angles[i])
    if(cnt[i][0][0]<a[0] and cnt[i][0][1]>a[1]):
      new_angles[i]=180-np.degrees(angles[i])
    if(cnt[i][0][0]>a[0] and cnt[i][0][1]>a[1]):
      new_angles[i]=360-np.degrees(angles[i])
    if(cnt[i][0][0]>a[0] and cnt[i][0][1]<a[1]):
      new_angles[i]=(np.degrees(angles[i]))*(-1)
    if(cnt[i][0][0]==a[0] and cnt[i][0][1]<a[1]):
      new_angles[i]=90
    if(cnt[i][0][0]<a[0] and cnt[i][0][1]==a[1]):
      new_angles[i]=180
    if(cnt[i][0][0]==a[0] and cnt[i][0][1]>a[1]):
      new_angles[i]=270
    if(cnt[i][0][0]>a[0] and cnt[i][0][1]==a[1]):
      new_angles[i]=0

  angles_analysis=list(range(0, 360, 5))

  final_angles=np.zeros(len(angles_analysis))
  for i in range(len(angles_analysis)):
    dist=np.zeros(len(new_angles))
    for j in range(len(new_angles)):
      dist[j]=abs(angles_analysis[i]-new_angles[j])
    final_angles[i]=np.argmin(dist)

  final_distances=np.zeros(len(final_angles))
  for i in range(len(final_angles)):
    final_distances[i]=distances[round(final_angles[i])]
  
  col_hist=0
  for a in range(len(final_distances)):
    worksheet.write(row_hist, col_hist, final_distances[a])
    col_hist+=1
  worksheet.write(row_hist, col_hist, img_names[shell])
  row_hist+=1



workbook.close()


## Extracción característica Textura

In [319]:
import random

In [359]:
# Acceso al directorio
directorio='C:/Users/Acer/Documents/Seminario/Shell_env - Editado/data_img'
img_names=os.listdir(directorio)

#Creando archivo xlsx
workbook = xlsxwriter.Workbook('texture_feature.xlsx')
worksheet = workbook.add_worksheet('texture_feature')

row_tex=0
for shell in range(0,10):
  print(shell)
  image_path = directorio + '/' + img_names[shell]
  img=cv2.imread(image_path)

  # Create a copy
  img2=img.copy()

  # Create empty mask.
  BW = np.zeros((img2.shape[0],img2.shape[1]),np.uint8)

  # Flood fill
  row=5
  column=5

  gray=cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY) #conversión a escala de grises
  ret,th=cv2.threshold(gray,30,255,cv2.THRESH_BINARY_INV) #binarizado inverso
  inv_th=cv2.bitwise_not(th)

  th2=inv_th.copy()
  h, w=th2.shape
  mask = np.zeros((h+2,w+2),np.uint8)

  cv2.floodFill(th2,mask,(row,column),255)
  inv=cv2.bitwise_not(th2)

  BW = inv_th|inv


  #Extracción de 200 parches aleatorios
  record_x=np.zeros(200)
  record_y=np.zeros(200)
  count_number=0
  while count_number<200:
    x=random.randint(0,279) #se eliminan los ultimos 20 pixeles de abajo para que el parche no sobrepase de las dimensiones de la imagen
    y=random.randint(0,379) #se eliminan los ultimos 20 pixeles de la derecha para que el parche no sobrepase de las dimensiones de la imagen
    if BW[x,y]==255:
      n=BW[x:x+20,y:y+20]
      if((n==255).sum()==400):
        record_x[count_number]=x
        record_y[count_number]=y
        count_number+=1

  wavelength=[5,10,15,20,25]
  orientation=[0,45,90,135]

  res_img=[]
  for patch in range(200):
    n=gray[round(record_x[patch]):round(record_x[patch])+20,round(record_y[patch]):round(record_y[patch])+20]
    for a in wavelength:
      for b in orientation:
        w = a #wavelength
        o = b / 180 * math.pi    # orientation in radian, and seems to run in opposite direction
        sigma = 0.5 * w * 1         # 1 == SpatialFrequencyBandwidth
        gamma = 0.5                          # SpatialAspectRatio
        shape = 1 + 2 * math.ceil(4 * sigma) # smaller cutoff is possible for speed
        shape = (shape, shape)
        gabor_filter_real = cv2.getGaborKernel(shape, sigma, o, w, gamma, psi=0)
        gabor_filter_imag = cv2.getGaborKernel(shape, sigma, o, w, gamma, psi=math.pi/2)

        gabor = cv2.filter2D(n, -1, gabor_filter_real) + 1j * cv2.filter2D(n, -1, gabor_filter_imag)
        mag = np.abs(gabor)
        phase = np.angle(gabor)

        mean_value=mag.mean()
        energy=((mag**2)/mag.size).sum()
        entropy=0
        for i in range(mag.shape[0]):
          for j in range(mag.shape[1]):
            if mag[i][j] != 0:
              entropy = entropy + ((abs(mag[i][j]/400))*math.log(abs(mag[i][j]/400)))
        entropy=-entropy
        res = [mean_value,energy,entropy]
        res_img.append(res)
        array = np.array(res_img).T
  
    col_hist=0
    for a in range(len(array)):
      worksheet.write(row_hist, col_hist, final_distances[a])
      col_hist+=1
    worksheet.write(row_hist, col_hist, img_names[shell])
    row_hist+=1
